In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
import os
import re
import csv
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.models import Sequential, load_model, model_from_json 
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dense,Activation,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,Dropout,AveragePooling2D,BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.metrics import accuracy_score
from tensorflow.python.client import device_lib

def walk(folder):
    """Walk through every files in a directory"""
    for dirpath, dirs, files in os.walk(folder):
        for filename in files:
            yield dirpath, filename


In [10]:
#Generate training, test set
path = '/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/Preprocessed_mfcc'
os.chdir(path)

In [11]:
keywords = []
for folder, filename in walk(path):
    keyword = re.split('/',folder)[-1]
    if keyword not in keywords:
        keywords.append(keyword)

In [12]:
keywords

['three',
 'tree',
 'up',
 'yes',
 'stop',
 'sheila',
 'wow',
 'two',
 'six',
 'zero',
 'left',
 'one',
 'nine',
 'on',
 'house',
 'off',
 'no',
 'marvin',
 'right',
 'seven',
 'go',
 'four',
 'dog',
 'eight',
 'bird',
 'happy',
 'cat',
 'down',
 'five',
 'bed']

In [13]:
temp = np.genfromtxt(folder+'/'+filename,delimiter=',')
print(temp)
print('shape:',np.shape(temp))

[[-1.5340e+01 -3.3836e+01  1.9575e+00 ...  3.3938e-02 -9.0522e-03
  -1.3330e-01]
 [-1.5248e+01 -3.3534e+01  1.6861e+00 ...  9.2822e-02  2.9942e-01
   1.2696e-01]
 [-1.5115e+01 -3.3489e+01  2.4876e+00 ... -9.7821e-02  4.2581e-02
   3.3344e-01]
 ...
 [-1.5059e+01 -3.3605e+01  2.8863e+00 ... -2.8762e-01 -2.9006e-01
   2.7579e-01]
 [-1.4960e+01 -3.3435e+01  2.5773e+00 ... -2.7075e-01  1.2803e-01
   2.6060e-01]
 [-1.4808e+01 -3.3136e+01  2.5948e+00 ... -1.1139e-01  2.3959e-02
   1.3450e-01]]
shape: (196, 14)


In [14]:
X_data = [];Y_data = []
for folder, filename in walk(path):
    temp = np.genfromtxt(folder+'/'+filename,delimiter=',')
    X_data.append(temp)
    Y_data.append(keywords.index(re.split('/',folder)[-1]))
X_data = np.expand_dims(X_data, axis=-1)
Y_data = np.expand_dims(Y_data, axis=-1)
np.save('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/X_data.npy',X_data)
np.save('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/Y_data.npy',Y_data)

In [15]:
#Load dataset
X_data = np.load('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/X_data.npy')
Y_data = np.load('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/Y_data.npy')
print('X_data len : %d, Y_data len : %d' % (len(X_data), len(Y_data)))

#Randomly choose 20% test data
tot_ix = range(len(Y_data))
test_ix = np.random.choice(tot_ix, int(len(Y_data)*0.2), replace=False)
test_ix = np.sort(test_ix,axis=0)
train_ix = list(set(tot_ix) - set(test_ix))

#Write test data index into csv files
test_ix = np.reshape(test_ix, test_ix.shape + (1,))
mat1 = test_ix
dataframe1 = pd.DataFrame(data=mat1.astype(int))
dataframe1.to_csv('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/20pertestset.csv', sep=',', header=False,
                  float_format = '%.2f', index = False)

#Load test index and convert to hot vector
test_index = pd.read_csv('/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/20pertestset.csv', header=None)
test_ix = test_index[0]
tot_ix = range(len(Y_data))
train_ix = list(set(tot_ix) - set(test_ix))
test_X = X_data[test_ix]
train_X = X_data[train_ix]
test_Y = Y_data[test_ix]
train_Y = Y_data[train_ix]
from keras.utils.np_utils import to_categorical
train_Y = to_categorical(train_Y, len(keywords))
test_Y = to_categorical(test_Y, len(keywords))


X_data len : 263, Y_data len : 263


In [16]:
np.shape(X_data)

(263, 196, 14, 1)

In [17]:
np.shape(temp)

(196, 14)

In [25]:
#Model
def generate_model_cnn(mode='words', pooling=0, layers=3, kernel=11):
    model = Sequential()
    print("=================================CNN=====================================")
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', input_shape=(196,14,1), activation = 'relu')) #add convolution layer
    model.add(AveragePooling2D(pool_size=(3,3), strides=2)) #pooling layer
    model.add(Conv2D(filters=30, kernel_size=(3,3), strides = 1, padding ='same', activation = 'relu'))
    model.add(AveragePooling2D(pool_size=(3,3), strides=None))
    model.add(Flatten())
    model.add(Dense(50, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.1))
    model.add(Dense(30, activation = 'softmax'))
    return model

def generate_model(mode='words', summary=True):
    model = generate_model_cnn(mode=mode, pooling=0)
    if summary:
        model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [26]:
save_path = '/content/gdrive/MyDrive/수학캡스톤_공유폴더/뼈대/'
callbacks_list  = [
    EarlyStopping( monitor = 'loss',  min_delta=0.001, patience=100,
                  verbose=1, mode='auto'),
    ModelCheckpoint(filepath = save_path+"/weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5",
                    monitor = 'val_loss',
                    save_best_only=True ),
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=7, verbose=1, min_delta=1e-4)]

model = generate_model(mode='words',summary=True)
history = model.fit(train_X, train_Y, epochs=100, batch_size=100, validation_data=(test_X,test_Y),callbacks=callbacks_list)

=================================CNN=====================================
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 196, 14, 30)       300       
_________________________________________________________________
average_pooling2d_2 (Average (None, 97, 6, 30)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 97, 6, 30)         8130      
_________________________________________________________________
average_pooling2d_3 (Average (None, 32, 2, 30)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                96050     
______________________________________________

In [27]:
def save_history_plot(hist,path):
    plt.ioff()
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()
    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    acc_ax.plot(hist.history['accuracy'], 'b', label='accuracy')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val accuracy')
    if 'top_k_categorical_accuracy' in hist.history:
        acc_ax.plot(hist.history['val_top_k_categorical_accuracy'], label='val top_k_accuracy')
        acc_ax.plot(hist.history['top_k_categorical_accuracy'], label='top_k_accuracy')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='upper right')
    plt.savefig(path)
    # plt.show()
    plt.close('all')
save_history_plot(history, save_path+"/model_history_img.png")
pd.DataFrame(history.history).to_csv(save_path+"/history.csv")

print("-- Evaluate --")
scores = model.evaluate(test_X, test_Y, batch_size=100)
print("%s: %.2f%%" %(model.metrics_names[0], scores[0]))
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

print("-- Predict --")
#output = model.predict_generator(my_test_batch_generator, workers = workers, use_multiprocessing = True)
output = model.predict(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

tt=[]
for i in range(len(output)):
    temp=[]
    temp.append(output[i].argsort()[::-1][:5] )
    temp.append(np.round(np.sort(output[i])[::-1][:5],6))
    tt.append(temp)
predict_df = pd.DataFrame(tt)

predicted_classes= output.argmax(axis=1)
answer_classes= test_Y.argmax(axis=1)
acc = accuracy_score(answer_classes, predicted_classes)
print('test accuracy :', acc)

# model save
model_json = model.to_json()
with open(save_path+"/model_acc_{:.4f}.json".format(acc), 'w') as json_file:
    json_file.write(model_json)

# weight save
model.save_weights(save_path +"/model_weight.h5")
model_json = model.to_json()
with open(save_path+"/model.json".format(acc), 'w') as json_file:
    json_file.write(model_json)

-- Evaluate --
1/1 [==============================] - 0s 17ms/step - loss: 5.3347 - accuracy: 0.0192
loss: 5.33%
accuracy: 1.92%
-- Predict --
test accuracy : 0.019230769230769232
